In [17]:
!pip install mp-api --upgrade
import numpy as np
from mp_api.client import MPRester
#from pymatgen.ext.matproj import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pymatgen.io.cif import CifParser


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [14]:
mp_key = "5KTRCxoFyWvECvvRVCS0OeflSzkuO7df"
mp_id = "mp-7575"

In [21]:
"""
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()
"""
#problème, donc j'ai fait avec le fichier cif

'\nwith MPRester(mp_key) as m:\n    prim_struc = m.get_structure_by_material_id(mp_id)\n    el_bs = m.get_bandstructure_by_material_id(mp_id)\n    el_dos = m.get_dos_by_material_id(mp_id)\n    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)\n    ph_dos = m.get_phonon_dos_by_material_id(mp_id)\nconv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()\nsymmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()\n'

In [34]:
parser = CifParser("LiZnN.cif")
structure = parser.get_structures()[0]

# Paramètres du réseau direct (cellule primitive)
lattice = structure.lattice
a, b, c = lattice.a, lattice.b, lattice.c
print("Vecteurs du réseau direct (cellule primitive):")
print(f"a = {a}, b = {b}, c = {c}")

# Obtenir les paramètres du réseau réciproque (cellule primitive)
reciprocal_lattice = lattice.reciprocal_lattice
a_r, b_r, c_r = reciprocal_lattice.a, reciprocal_lattice.b, reciprocal_lattice.c
print("\nVecteurs du réseau réciproque:")
print(f"a* = {a_r}, b* = {b_r}, c* = {c_r}")


# Structure conventionnelle
analyzer = SpacegroupAnalyzer(structure)
conventional_structure = analyzer.get_conventional_standard_structure()

# Cellule conventionnelle
conventional_lattice = conventional_structure.lattice
a, b, c = conventional_lattice.a, conventional_lattice.b, conventional_lattice.c
print("\nParamètres de la cellule conventionnelle:")
print(f"a = {a}, b = {b}, c = {c}")

# Réseau réciproque liés aux vecteurs de base de la cellule conventionnelle
reciprocal_lattice = conventional_lattice.reciprocal_lattice
a_r, b_r, c_r = reciprocal_lattice.a, reciprocal_lattice.b, reciprocal_lattice.c
print("\nParamètres du réseau réciproque liés à la cellule conventionnelle:")
print(f"a* = {a_r}, b* = {b_r}, c* = {c_r}")

#Type de maille
maille_type = analyzer.get_lattice_type()

print("\nType de maille:", maille_type)

#Système cristallin
crystal_system = analyzer.get_crystal_system()
print("\nSystème cristallin:", crystal_system)

#Groupe ponctuel:
point_group_symbol = analyzer.get_space_group_symbol()
print("\nGroupe ponctuel:", point_group_symbol)

Vecteurs du réseau direct (cellule primitive):
a = 3.439929088105139, b = 3.4399290881051385, c = 3.4399290881051385

Vecteurs du réseau réciproque:
a* = 2.2370516321341047, b* = 2.2370516321341056, c* = 2.2370516321341056

Paramètres de la cellule conventionnelle:
a = 4.86479437, b = 4.86479437, c = 4.86479437

Paramètres du réseau réciproque liés à la cellule conventionnelle:
a* = 1.2915623620037173, b* = 1.2915623620037173, c* = 1.2915623620037173

Type de maille: cubic

Système cristallin: cubic

Groupe ponctuel: F-43m
